In [ ]:
import requests
import json
import pandas as pd
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import re
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook

disable_warnings(InsecureRequestWarning)

base_url = "https://viperdb.scripps.edu"

all_tnums_url = "/services/tnumber_index.php?serviceName=tnumbers"
all_genuses_url = "/services/genus_index.php?serviceName=genus"
all_families_url = "/services/family_index.php?serviceName=families"

tnum_members_url = "/services/tnumber_index.php?serviceName=tnumber_members&tnumber="
genus_members_url = "/services/genus_index.php?serviceName=genus_members&genus="
family_members_url = "/services/family_index.php?serviceName=family_members&family="

rcsb_url = 'https://data.rcsb.org/rest/v1/core/entry/'
vdb_url = "https://viperdb.scripps.edu/"
radius_url = "services/biodata.php?serviceName=radius_formatted&VDB="

def getrcsb(c):
    res = requests.get(rcsb_url + c, verify=False)
    return res
def getrad(c):
    res = requests.get(vdb_url + radius_url + c, verify=False)
    return res

def tnum_url(n):
    return base_url + tnum_members_url + str(n)
def genus_url(n):
    return base_url + genus_members_url + str(n)
def family_url(n):
    return base_url + family_members_url + str(n)

#sort list of dictionaries by resolution and take the n lowest
def lowres_dictlist(dictlist, n):
    lowres = {}
    for k in dictlist.keys():
        sortres = sorted(dictlist[k], key=lambda d: (d['resolution'] is None, d['resolution']))
        lowres[k] = sortres[:n]
    return lowres
def dictlist(dictlist):
    res = {}
    for k in dictlist.keys():
        sortres = sorted(dictlist[k], key=lambda d: (d['resolution'] is None, d['resolution']))
        res[k] = sortres
    return res

In [ ]:
families_res = requests.get((base_url + all_families_url), verify=False)
families = json.loads(families_res.text)
family_members = {}
for i in range(len(families)):
    family = families[i]["family"]
    members_res = requests.get(family_url(family), verify=False)
    family_members[family] = json.loads(members_res.text)
family_dict = dictlist(family_members)

In [10]:
dfs = []
for fam, vs in family_dict.items():
        df_dict = {}
        headers = ['tnumber', 'id', 'genome', 'weight', 'radius', 'atoms', 'mers', 'mer_weight_maxs', 'mer_weight_mins']
        for h in headers:
            df_dict[h] = []
        
        for virus in vs:
            pdb_id = virus['entry_id']
            df_dict['id'].append(pdb_id)
            
            df_dict['tnumber'].append(virus['tnumber'])
            df_dict['genome'].append(virus['genome'])

            try:
                rcsb_res = getrcsb(pdb_id)

                resdict = eval(rcsb_res._content.decode())

                df_dict['weight'].append(resdict['rcsb_entry_info']['molecular_weight'])

                df_dict['atoms'].append(resdict['rcsb_entry_info'][ 'deposited_atom_count'])

                df_dict['mers'].append(resdict['rcsb_entry_info']['deposited_polymer_monomer_count'])

                df_dict['mer_weight_maxs'].append(resdict['rcsb_entry_info']['polymer_molecular_weight_maximum'])

                df_dict['mer_weight_mins'].append(resdict['rcsb_entry_info']['polymer_molecular_weight_minimum'])

                #get avg radius
                vdb_res = getrad(pdb_id)
                pres = vdb_res._content.decode().split('Ave')[-1].split(': ')[1]

                rad = re.split(r'\D+',pres)[0]
                df_dict['radius'].append(rad)
            except Exception as e:
                #pass
                print(pdb_id, e)
        try:
            df = pd.DataFrame.from_dict(df_dict, orient='index')
            df = df.transpose()
            dfs.append(df)
        except Exception as e:
            print(e)

2stv 'rcsb_entry_info'
amv 'rcsb_entry_info'
ccmv_swln_2 'rcsb_entry_info'
ccmvswln1 'rcsb_entry_info'
1e6t 'rcsb_entry_info'
1e7x 'rcsb_entry_info'
1gkw 'rcsb_entry_info'
1he6 'rcsb_entry_info'
1kuo 'rcsb_entry_info'
1h8j 'rcsb_entry_info'
1gkv 'rcsb_entry_info'
2q23 'rcsb_entry_info'
2q25 'rcsb_entry_info'
1l0t 'rcsb_entry_info'
6mf7 'rcsb_entry_info'
6dij 'rcsb_entry_info'
3j69 'rcsb_entry_info'
4agy 'rcsb_entry_info'
3j6a 'rcsb_entry_info'
3c5b 'rcsb_entry_info'
hrtvc1 'rcsb_entry_info'
hrtvc2 'rcsb_entry_info'
3iyu 'rcsb_entry_info'
3n09 'rcsb_entry_info'
rcmv 'rcsb_entry_info'
3p8q 'rcsb_entry_info'


In [13]:
dfs[-1]

,tnumber,id,genome,weight,radius,atoms,mers,mer_weight_maxs,mer_weight_mins
0,1,4oq8,ssRNA,24.68,87,1946,181,17.53,0.57
1,1,4oq9,ssRNA,367.95,87,29296,2715,17.53,0.57
2,1,1a34,ssRNA,24.22,87,1763,179,17.53,3.02
3,1,4nia,ssRNA,365.93,87,28403,2715,17.53,0.57


In [14]:
xl_file = 'families.xlsx'
wb = Workbook()
ws =  wb.active
wb.save(filename = xl_file)

book = load_workbook(xl_file)
writer = pd.ExcelWriter(xl_file, engine = 'openpyxl')
writer.book = book
for i, df in enumerate(dfs):
    fam = families[i]['family']
    df.to_excel(writer, sheet_name = fam, index = False)
del book['Sheet']
book.save(xl_file)

In [ ]:
'''
tnums_res = requests.get((base_url + all_tnums_url), verify=False)
tnums = json.loads(tnums_res.text)
genuses_res = requests.get((base_url + all_genuses_url), verify=False)
genuses = json.loads(genuses_res.text)
families_res = requests.get((base_url + all_families_url), verify=False)
families = json.loads(families_res.text)

tnum_members = {}
for i in range(len(tnums)):
    tnum = tnums[i]["tnumber"]
    members_res = requests.get(tnum_url(tnum), verify=False)
    tnum_members[tnum] = json.loads(members_res.text)
    
n_structures = 100
tnum_lowres = lowres_dictlist(tnum_members, n_structures)

#make dict with all members for each tnum/genus/family
tnum_members = {}
for i in range(len(tnums)):
    tnum = tnums[i]["tnumber"]
    members_res = requests.get(tnum_url(tnum), verify=False)
    tnum_members[tnum] = json.loads(members_res.text)

genus_members = {}
for i in range(len(genuses)):
    genus = genuses[i]["genus"]
    members_res = requests.get(genus_url(genus), verify=False)
    genus_members[genus] = json.loads(members_res.text)

family_members = {}
for i in range(len(families)):
    family = families[i]["family"]
    members_res = requests.get(family_url(family), verify=False)
    family_members[family] = json.loads(members_res.text)
family_lowres = lowres_dictlist(family_members, n_structures)
    
#make the dictlists only contain the n lowest res structures
n_structures = 5
tnum_lowres = lowres_dictlist(tnum_members, n_structures)
genus_lowres = lowres_dictlist(genus_members, n_structures)
family_lowres = lowres_dictlist(family_members, n_structures)

#get families containing each FSF
file_url = 't2.txt'
fsf_families = {}
with open(file_url, 'r') as f:
    lines = f.readlines()
    for l in lines:
        l = l.split()
        fams = [x.replace(',', '') for x in l[1:]]
        fsf_families[l[0]] =fams 

#make dict of each FSF to n lowest res structures from each associated family
fsf_lowres = {}
for k, v in fsf_families.items():
    members = {}
    for f in v:
        try: 
            members[f] = family_lowres[f]
        except:
            pass
    fsf_lowres[k] = members
    
fsf_one = []
klist = list(fsf_lowres.keys())
ids = []
for i in range(len(list(fsf_lowres.values()))):
    try:
        added = False
        vlist = list(list(fsf_lowres.values())[i].values())[0]
        for v in vlist:
            if v['entry_id'] not in ids:
                fsf_one.append(v)
                ids.append(v['entry_id'])
                added = True
                break
        if not added:
            fsf_one.append(vlist[0])
            added = True
        if added: fsf_one[-1]["fold"] = klist[i]
    except Exception as e:
        #print(e)
        pass
'''